In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import datetime as dt
path = 'C:\\Users\\hchb1\\Downloads\\OneDrive_2024-08-22\\ETL data\\'
df = pd.read_json(path + os.listdir(path)[1]+ "\\data.json", lines = True)
# EDA 
for i in df.columns:
    print("Checking", i, "column")
    print('tong so luong', df[i].nunique()) # in tổng số lượng giá trị unique trong cột
    print(df[i].unique()) # in ra giá trị unique trong cột

# Các chỉ số gồm: xem nhiều nhất ở cái gì (most watch), tổng view (total view), lần sử dụng gần nhất (recency)
def most_watch (df):
    user_statistic = df[['receive_time', 'device_id', 'ItemName', 'AppId']].groupby(['device_id', 'ItemName', 'AppId']).count().reset_index()
    user_statistic = user_statistic[user_statistic['device_id'] != '']
    user_statistic.rename(columns= {'receive_time' : 'total_wach'}, inplace=True)
    user_statistic['Rank'] = user_statistic.groupby(['device_id'])['total_wach'].rank(method= 'dense', ascending=False)
    user_statistic = user_statistic[user_statistic['Rank'] == 1.0]
    Most_watch['row_number'] = Most_watch.groupby("device_id").cumcount() + 1
    Most_watch = Most_watch[Most_watch['row_number'] == 1]
    Most_watch = Most_watch[['device_id', 'Most_watch']]
    return Most_watch

def total_view_cus (df):
    total_view = df[['device_id', 'receive_time']].groupby((['device_id'])).count().reset_index().sort_values('device_id').rename(columns= {'receive_time' : 'Total_view'})
    total_view = total_view[total_view['device_id'] !=''].sort_values('Total_view')
    return total_view

def recency_cus (df):
    df['current date'] = '2024-01-01'
    df['timestamp'] = pd.to_datetime(df['receive_time'])
    df['current date'] = pd.to_datetime(df['current date'])
    df['recency'] = df['current date'] - df['timestamp']
    df_recency = df.groupby('device_id')['recency'].min().reset_index()
    return df_recency
    
def output_table_join (df):
    Table_OLAP = Most_watch.merge(total_view, left_on = 'device_id', right_on= 'device_id', how = 'inner').merge(df_recency, left_on = 'device_id', right_on ='device_id', how = 'inner')
    return Table_OLAP

In [ ]:
import mysql.connector

def import_to_database(user, password, database, port, data):
    try:
        cnx = mysql.connector.connect(user=user, password=password, database=database, port=port)
        cursor = cnx.cursor()
        sql = """INSERT INTO ETL_table (
            device_id,
            Most_watch,
            total_view,
            df_recency,
        ) VALUES (%s, %s, %s, %s)"""
        cursor.executemany(sql, data.values.tolist())
        cnx.commit()
        print("Import Successfully")
    except mysql.connector.Error as e:
        print("Error:", e)
        # Handle the error, e.g., retry the query or log the error
    finally:
        if cnx:
            cnx.close()

# Example usage:
user = 'root'
password = ''
database = 'import_python'
port = 3306
data = Table_OLAP  # Assuming overview is your DataFrame

import_to_database(user, password, database, port, data)